In [281]:
import pandas as pd
import gradio as gr
import nltk
import numpy as np

In [282]:
df_descripteur = pd.read_csv('freq_poids_porter.csv')

In [283]:
df_inverse = pd.read_csv('df_poids.csv').drop('Unnamed: 0', axis=1)

In [284]:
# demo = gr.Interface(fn=search, inputs="number", outputs="numpy").queue()
# demo.launch()

In [285]:
def search_document_descripteur(document):
        new_df = df_descripteur[df_descripteur['Document'] == document]
        new_df.drop(['Document'], axis=1, inplace=True)
        
        inverse = df_inverse.iloc[int(document)].to_frame()
        inverse = inverse[(inverse.T != 0.000000).all()]
        inverse = inverse.reset_index(level=0)
        inverse["Terme"] = inverse["index"]
        inverse["Poid"] = inverse[5]
        inverse.drop([5], axis=1, inplace=True)
        inverse.drop(['index'], axis=1, inplace=True)

        return [new_df, inverse]

In [286]:
search_document_descripteur(5)[1]

C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Terme,Poid
0,edit,0.154349
1,studi,0.070184
2,need,0.083358
3,detail,0.118908
4,work,0.083654
...,...,...
94,effect,0.096608
95,store,0.168212
96,chapter,0.147169
97,devot,0.165555


In [287]:
def search_document_inverse(phrase):
        ExpReg = nltk. RegexpTokenizer('(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%?|\w+(?:\-\w+)*')
        termes = ExpReg.tokenize(phrase)
        termes = [t.lower() for t in termes]
        MotsVides = nltk.corpus.stopwords.words('english')
        Porter = nltk.PorterStemmer()
        TermesSansMotsVides_p = [Porter.stem(terme) for terme in termes if terme.lower() not in MotsVides]
        
        new_df = df_inverse[TermesSansMotsVides_p]
        new_df = new_df[(new_df.T != 0.000000).any()]
        new_df = new_df.reset_index(level=0)

        desc_df = df_descripteur.loc[df_descripteur['Terme'].isin(TermesSansMotsVides_p) ]

        #new_df.drop(['document'], axis=1, inplace=True)
        return new_df, desc_df

In [288]:
search_document_inverse('What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles?')

(     index   problem   concern      make  descript  titl  difficulti  \
 0        3  0.000000  0.000000  0.000000       0.0   0.0    0.326115   
 1        4  0.000000  0.000000  0.000000       0.0   0.0    0.000000   
 2        6  0.252204  0.000000  0.332710       0.0   0.0    0.000000   
 3        7  0.000000  0.201570  0.000000       0.0   0.0    0.000000   
 4        8  0.252204  0.000000  0.000000       0.0   0.0    0.000000   
 ..     ...       ...       ...       ...       ...   ...         ...   
 981   1450  0.000000  0.000000  0.000000       0.0   0.0    0.000000   
 982   1453  0.252204  0.000000  0.000000       0.0   0.0    0.543524   
 983   1454  0.000000  0.251962  0.249533       0.0   0.0    0.000000   
 984   1457  0.000000  0.000000  0.332710       0.0   0.0    0.000000   
 985   1458  0.252204  0.000000  0.000000       0.0   0.0    0.000000   
 
        involv  automat  retriev  articl  approxim  titl  
 0    0.000000      0.0      0.0     0.0  0.000000   0.0  
 1  

In [289]:
with gr.Blocks() as app:
        gr.Markdown("main app")
        with gr.Tab("Main Searches"):
                with gr.Row():
                        with gr.Column():
                                gr.Markdown("Search by Term")
                                search_input = gr.Textbox(label="Search for terms")
                                search_button = gr.Button(label="Search by Term")
                with gr.Row():
                        with gr.Column():
                                gr.Markdown("Search by Document")
                                search_document = gr.Number()
                                search_document_button = gr.Button(label="Search by Document")
                with gr.Row():
                        with gr.Column():
                                gr.Markdown("Descripteur")
                                search_output_descripteur = gr.Numpy()
                        with gr.Column():
                                gr.Markdown("Inverse")
                                search_output_inverse = gr.Numpy()
        with gr.Tab("Boolean Search"):
                gr.Markdown("Boolean Search")
                search_boolean = gr.Textbox(label="Boolean search")
                search_boolean_button = gr.Button(label="Search")
                search_boolean_output = gr.Numpy()
        with gr.Tab("DBScan"):
                gr.Markdown("DBScan")
        with gr.Tab("Naive Bayes"):
                gr.Markdown("Naive Bayes")
        search_button.click(search_document_inverse, inputs = search_input, outputs = [search_output_inverse, search_output_descripteur])
        search_document_button.click(search_document_descripteur, inputs = search_document, outputs = [search_output_descripteur, search_output_inverse])

app.launch()

Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.
